# buy-open-sell-close

On the first trading day of each week, buy on the open, then sell on the close

In [1]:
import pandas as pd
import datetime
import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

Some global data

In [2]:
symbol = 'SPY'
capital = 10000
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime.now()

Timeseries

In [3]:
# fetch timeseries, select
ts = pf.fetch_timeseries(symbol)
ts = pf.select_tradeperiod(ts, start, end, use_adj=False)

# add calendar columns
ts = pf.calendar(ts)

# finalize timeseries
ts, start = pf.finalize_timeseries(ts, start)

# create tradelog and daily balance objects
tlog = pf.TradeLog(symbol)
dbal = pf.DailyBal()

In [4]:
ts

,high,low,open,close,volume,adj_close,dotw,dotm,doty,month,first_dotw,first_dotm,first_doty,last_dotw,last_dotm,last_doty
date,,,,,,,,,,,,,,,,
2015-01-02,206.88,204.18,206.38,205.43,121465900.00,182.15,4,2,2,1,False,True,True,True,False,False
2015-01-05,204.37,201.35,204.17,201.72,169632600.00,178.86,0,5,5,1,True,False,False,False,False,False
2015-01-06,202.72,198.86,202.09,199.82,209151400.00,177.18,1,6,6,1,False,False,False,False,False,False
2015-01-07,202.72,200.88,201.42,202.31,125346700.00,179.38,2,7,7,1,False,False,False,False,False,False
2015-01-08,206.16,203.99,204.01,205.90,147217800.00,182.57,3,8,8,1,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-22,394.07,389.97,390.03,392.59,73778600.00,392.59,0,22,81,3,True,False,False,False,False,False
2021-03-23,393.46,388.66,391.91,389.50,90686600.00,389.50,1,23,82,3,False,False,False,False,False,False
2021-03-24,392.75,387.47,391.00,387.52,97588600.00,387.52,2,24,83,3,False,False,False,False,False,False


Algorithm

In [5]:
pf.TradeLog.cash = capital

# loop through timeseries
for i, row in enumerate(ts.itertuples()):

    date = row.Index.to_pydatetime()
    end_flag = pf.is_last_row(ts, i)

    # buy open, sell close
    if row.first_dotw:
        tlog.buy(date, row.open)
        tlog.sell(date, row.close)

    # record daily balance
    dbal.append(date, row.high, row.low, row.close)

Retrieve logs

In [6]:
tlog = tlog.get_log()
dbal = dbal.get_log(tlog)

In [7]:
tlog.tail()

,entry_date,entry_price,exit_date,exit_price,pl_points,pl_cash,qty,cumul_total,direction,symbol
320,2021-02-22,387.06,2021-02-22,387.03,-0.03,-0.81,27,477.36,LONG,SPY
321,2021-03-01,385.59,2021-03-01,389.58,3.99,107.73,27,585.09,LONG,SPY
322,2021-03-08,384.66,2021-03-08,381.72,-2.94,-79.38,27,505.71,LONG,SPY
323,2021-03-15,394.33,2021-03-15,396.41,2.08,54.08,26,559.79,LONG,SPY
324,2021-03-22,390.03,2021-03-22,392.59,2.56,69.12,27,628.91,LONG,SPY


Get stats

In [8]:
stats = pf.stats(ts, tlog, dbal, capital)

Summary

In [9]:
pf.summary(stats)

,strategy
annual_return_rate,0.98
max_closed_out_drawdown,-13.98
best_month,6.04
worst_month,-5.08
sharpe_ratio,0.21
sortino_ratio,0.12
monthly_std,1.45
annual_std,6.95
